In [4]:
import cv2 as cv
import numpy as np


net = cv.dnn.readNetFromTensorflow("graph_opt.pb")


BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

image_path = "image2.jpg"  
frame = cv.imread(image_path)

frameHeight, frameWidth = frame.shape[:2]
net.setInput(cv.dnn.blobFromImage(frame, 1.0, (368, 368), (127.5, 127.5, 127.5), swapRB=True, crop=False))
out = net.forward()

points = []
for i in range(len(BODY_PARTS)):
    heatMap = out[0, i, :, :]
    _, conf, _, point = cv.minMaxLoc(heatMap)
    x = int((frameWidth * point[0]) / heatMap.shape[1])
    y = int((frameHeight * point[1]) / heatMap.shape[0])
    points.append((x, y) if conf > 0.2 else None)

for pair in POSE_PAIRS:
    partFrom, partTo = pair
    idFrom, idTo = BODY_PARTS[partFrom], BODY_PARTS[partTo]
    if points[idFrom] and points[idTo]:
        cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
        cv.circle(frame, points[idFrom], 3, (0, 0, 255), -1)
        cv.circle(frame, points[idTo], 3, (0, 0, 255), -1)

cv.imshow('Pose Estimation', frame)
cv.imwrite('output_image.jpg', frame)  
cv.waitKey(0)
cv.destroyAllWindows()
